In [3]:
import requests
import os

Day = 10

# get file from website using private session key stored in enviromental variables
r = requests.get(
            f'https://adventofcode.com/2023/day/'+str(Day)+'/input',
            cookies={'session': os.getenv('AdventSessionKey')}
)

# read r.text
text_array = r.text.strip().split('\n')


In [4]:
def convert(text_array):
    # Define the directions for each symbol
    directions = {
        "|": ["N", "S"],
        "-": ["E", "W"],
        "L": ["N", "E"],
        "J": ["N", "W"],
        "7": ["S", "W"],
        "F": ["S", "E"]
    }

    # Define the opposite directions
    opposite_directions = {
        "N": "S",
        "E": "W",
        "S": "N",
        "W": "E"
    }

    # Define the movement for each direction
    movement = {"N": (-1, 0), "E": (0, 1), "S": (1, 0), "W": (0, -1)}

    # Find the starting position
    start_row, start_col = next((i, j) for i, row in enumerate(text_array) for j, cell in enumerate(row) if cell == "S")
    
    # Try each direction one by one
    for start_direction in ["N", "E", "S", "W"]:

        # Initialize the number array with the same size as the text array
        number_array = [[None]*len(row) for row in text_array]
        
        # Assign 0 to the starting position
        number_array[start_row][start_col] = (0, None)
        
        # Initialize the symbol array with the same size as the text array
        symbol_array = [['.']*len(row) for row in text_array]

         # Assign S to the starting position
        symbol_array[start_row][start_col] = 'S'

        # Initialize the path array with the starting position
        path = [(start_row, start_col)]

        # Start from the 'S' symbol and the current direction
        row, col, direction, distance = start_row, start_col, start_direction, 0

        while True:
            # Move to the next position based on the direction
            d_row, d_col = movement[direction]
            row += d_row
            col += d_col

            # Check if we're out of bounds or have looped back to the 'S' symbol
            if not(0 <= row < len(text_array) and 0 <= col < len(text_array[0])) or (row == start_row and col == start_col):
                break

            # Check the symbol at the new position
            symbol = text_array[row][col]
            if symbol in directions and opposite_directions[direction] in directions[symbol]:
                # Update the distance and the number array
                distance += 1
                number_array[row][col] = (distance, direction)

                # Add the new position to the path
                path.append((row, col))

                # Update the symbol array
                symbol_array[row][col] = symbol

                # Update the direction to the other connecting direction for the symbol
                direction = next(d for d in directions[symbol] if d != opposite_directions[direction])

        # Check if we have looped back to the 'S' symbol
        if row == start_row and col == start_col:
            return number_array, symbol_array, path, distance

    # Return Nones if it failed
    return None, None, None, None

# Print the result
number_array, symbol_array, path, intersection_distance = convert(text_array)
#for row in number_array:
#    print([cell[0] if cell is not None else None for cell in row])
print("The distance at the point of intersection is:", int((intersection_distance+1) / 2))
# 6867

The distance at the point of intersection is: 6867


In [11]:
def count_total_periods(symbol_array):
    # Define the north-facing symbols (thanks to Reddit hint)
    # I cheated by adding S to this list after looking at my part 1 symbol_array 
    north_facing_symbols = ["|", "L", "J", "S"]

    # Initialize the total count
    total_count = 0

    # Scan each line
    for row in symbol_array:
        # Initialize the count and the tracking variable for the current line
        count = 0
        tracking = False

        # Scan each symbol in the line
        for symbol in row:
            if symbol in north_facing_symbols:
                # If the symbol is a north-facing symbol, add the count to the total count, reset the count, and flip the tracking variable
                total_count += count
                count = 0
                tracking = not tracking
            elif symbol == '.' and tracking:
                # If the symbol is a period and the tracking variable is ON, increment the count
                count += 1

    return total_count

# Print the result
total_periods = count_total_periods(symbol_array)
print("The count of periods inside the loop is:", total_periods)


The count of periods inside the loop is: 595


In [12]:
def count_lattice_points(path):
    # Initialize the counts
    B = len(path)  # All points on the path are boundary points
    I = 0  # Start with no interior points

    # Get the min and max coordinates of the path
    min_row = min(point[0] for point in path)
    max_row = max(point[0] for point in path)
    min_col = min(point[1] for point in path)
    max_col = max(point[1] for point in path)

    # Check all points within the bounding box of the path
    for row in range(min_row + 1, max_row):
        for col in range(min_col + 1, max_col):
            if (row, col) not in path and is_inside_polygon((row, col), path):
                # If the point is not on the path, it's an interior point
                I += 1

    return B, I

# Calculate the number of boundary and interior points
B, I = count_lattice_points(path)

print("Number of interior points:", I)


Number of interior points: 595


In [10]:
def is_inside_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(1, n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def count_interior_periods(symbol_array, path):
    # Initialize the count
    count = 0

    # Scan each cell in the symbol array
    for i, row in enumerate(symbol_array):
        for j, cell in enumerate(row):
            if cell == '.' and is_inside_polygon((i, j), path):
                # If the cell is a period and is inside the path, increment the count
                count += 1

    return count

# Calculate the number of interior periods
interior_periods = count_interior_periods(symbol_array, path)
print("Number of interior periods:", interior_periods)

Number of interior periods: 595
